In [4]:
# Facial image emotion recognition using CNN
# Target classes: Angry, Disgusted, Fearful, Happy, Sad, Surprised, Neutral

import tensorflow as tf

# DATASET
# files in dataset/test and dataset/train
# subfolders: angry, disgusted, fearful, happy, sad, surprised, neutral

# load dataset
train_dir = 'dataset/train'
test_dir = 'dataset/test'

In [5]:
# Test data
# Check for the number of images in each class and the size of the images
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# List of classes
classes = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']

# Number of images in each class
print('Number of training images in each class:')
for c in classes:
    path = os.path.join(train_dir, c)
    print(f'{c}: {len(os.listdir(path))} images')
    
print('\nNumber of test images in each class:')
for c in classes:
    path = os.path.join(test_dir, c)
    print(f'{c}: {len(os.listdir(path))} images')
    
# Image size
img = cv2.imread('dataset/train/angry/im0.png')
print(f'\nImage size: {img.shape}')

Number of training images in each class:
angry: 3995 images
disgusted: 436 images
fearful: 4097 images
happy: 7215 images
sad: 4830 images
surprised: 3171 images
neutral: 4965 images

Number of test images in each class:
angry: 958 images
disgusted: 111 images
fearful: 1024 images
happy: 1774 images
sad: 1247 images
surprised: 831 images
neutral: 1233 images

Image size: (48, 48, 3)


In [4]:
# CNN MODEL
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# DATA PREPROCESSING
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)  # Stelle sicher, dass deine Bilder im JPEG-Format sind
    img = tf.image.resize(img, [48, 48])
    return img / 255.0

def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def prepare_for_training(ds, batch_size=32, cache=True, shuffle_buffer_size=1000):
    # Diese Funktion konfiguriert das Dataset für verbesserte Leistung
    if cache:
        ds = ds.cache()
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

def process_labels(labels):
    # Umwandlung von Textlabels in One-Hot-kodierte Labels
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    return labels

train_ds = tf.data.Dataset.list_files(os.path.join(train_dir, '*/*.jpg'))
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Labels verarbeiten
labels = train_ds.map(lambda x, y: y)
labels = process_labels(labels)
train_ds = train_ds.map(lambda x, y: (x, process_labels(tf.expand_dims(y, axis=0))))

train_ds = prepare_for_training(train_ds)

test_ds = tf.data.Dataset.list_files(os.path.join(test_dir, '*/*.jpg'))
test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Labels verarbeiten
test_ds = test_ds.map(lambda x, y: (x, process_labels(tf.expand_dims(y, axis=0))))
test_ds = prepare_for_training(test_ds, cache=False)



2024-05-04 15:01:35.475861: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/20


2024-05-04 15:01:37.270178: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


314/898 [=========>....................] - ETA: 1:10 - loss: 1.7794 - accuracy: 0.2716

In [ ]:
# TRAIN MODEL
train_steps = len(os.listdir(train_dir)) // 32
test_steps = len(os.listdir(test_dir)) // 32

history = model.fit(
    train_ds,
    epochs=20,
    steps_per_epoch=train_steps,
    validation_data=test_ds,
    validation_steps=test_steps
)

In [ ]:
# PLOT TRAINING AND VALIDATION ACCURACY
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# PLOT TRAINING AND VALIDATION LOSS
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 2])
plt.legend(loc='upper right')
plt.show()

# SAVE MODEL
model.save('emotion_recognition_model.h5')
print('Model saved as emotion_recognition_model.h5')

# TEST MODEL
# Load model
model = tf.keras.models.load_model('emotion_recognition_model.h5')

# Load test image
img = cv2.imread('dataset/test/angry/im0.png')
img = cv2.resize(img, (48, 48))
img = np.reshape(img, [1, 48, 48, 3])

# Predict emotion
prediction = model.predict(img)
emotion = classes[np.argmax(prediction)]
print(f'Predicted emotion: {emotion}')

# Display image
plt.imshow(cv2.cvtColor(img[0], cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# Display prediction
plt.bar(classes, prediction[0])
plt.ylabel('Probability')
plt.show()